In [11]:
#
# TODO: Extraer data desde base de datos
# TODO: Exportar el resultado del prompt
# TODO: Importar prompt desde un archivo .md
# * Se puede leer el archivo, traer el texto y reemplazar texto como {pais} usando .replace y asignando las variables definidas

In [12]:
from google import genai
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR

### Data básica

In [24]:
marca = "Hero"
modelo = "Hunk 125R"
año = 2025
país = "Colombia"
tipo = "Naked"
code = "CO2961-hero-hunk-125-r"

### Prompt a enviar

In [14]:
from utils.prompt_utils import *

prompt_from_file = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "knowledge_base_template.md"))
prompt = replace_variables(prompt_from_file, {
    "{marca}": marca,
    "{modelo}": modelo,
    "{año}": str(año),
    "{país}": país,
    "{tipo}": tipo
})

### Usando Gemini

In [15]:
client = genai.Client() # La API Key se lee automaticamente desde variables de entorno

# Llamada a la API
interaction_con = client.interactions.create(
    model='gemini-3-pro-preview',
    input=prompt,
    tools=[{'type': 'google_search'}]  # Habilita Google Search
)

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_18908\417860456.py:4: UserWarning: Interactions usage is experimental and may change in future versions.
  interaction_con = client.interactions.create(


### Output de la API

In [18]:
final_output = ""
for output in interaction_con.outputs:
    if hasattr(output, "text"):
        final_output += output.text

### Exportar OUTPUT

In [25]:
with open(f"{SRC_DIR}/data/output_gemini/{code}-{país}.md", "w", encoding="utf-8") as f:
    f.write(final_output)

### Usando Replicate

In [ ]:
# client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

# output = client.run(
#     ref = "google/gemini-3-pro",
#     input={
#         "prompt": prompt
#     },
# )

# print("\n--- RESPUESTA ---\n")
# if isinstance(output, list):
#     print("".join([str(x) for x in output]))
# else:
#     print(output)